### Import Libraries

In [ ]:
from setupGenerators import *
from unet_keras import unet
from keras.callbacks import ModelCheckpoint

### Training Parameters

In [ ]:
path = "../../data/groundcover2016/maize/(2048, 1152)"
# Firm Training variables
width = 2048
height = 1152
input_shape = (width,height,3)
# Variables to tweak
steps_per_epoch = 100
epochs = 10
batch_size = 20

### Load in Model and Generator

In [ ]:
# load in model 
model = unet(2,input_shape)
# initialize generator
generator = generateData(path = path,save_dir = False,batch_size = batch_size)

### Train Model

In [ ]:
model_checkpoint = ModelCheckpoint(path + '/train/unet_train.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(generator,steps_per_epoch = steps_per_epoch,epochs = epochs,callbacks =[model_checkpoint])